In [11]:
import torch
import os
from torchvision import datasets, models, transforms
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import glob
import copy
import neptune
#import splitfolders # Библиотека для разделения файлов картинок на train test
import matplotlib.gridspec as gridspec
cudnn.benchmark = True

In [12]:
run = neptune.init_run(
    project="neas1231/Neas1231",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmY2VjMGIzOS01ZjI1LTQ1MTItODQxYi0zMjIyOWIwYWI0MzIifQ==",
)

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((128 , 128)),   
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((128 , 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\datasets\dataset'
run["config/data_dir"] = data_dir

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
              for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) 
              for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

https://app.neptune.ai/neas1231/Neas1231/e/NEAS-113


In [13]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=14):
    since = time.time()
    #Сохраняем  лучшие веса 
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 99
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)


        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            # перенос  вычислений на куду 
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                
                optimizer.zero_grad()

                #
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    acc = (torch.sum(preds == labels.data)) / len(inputs)

                    # считаем градиенты тольок если трейн данные 
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # стасистика  
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # сохраняем не только лучшие веса , но и акураси 
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Loss: {best_loss:4f}')
    

    model.load_state_dict(best_model_wts)
    return model

In [14]:
model_ft = models.efficientnet_b4(pretrained=True) 
num_ftrs = model_ft.classifier[1].in_features


# cuda
model_ft = model_ft.to(device)

model_ft.classifier = nn.Sequential(*list(model_ft.classifier.children()))[:-1]


criterion = nn.CrossEntropyLoss()





# оптимайзер  с лернинг рейт 
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# редактируем лернинг рейт каждые 7 шагов 
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
model_ft.parameters

<bound method Module.parameters of EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
       

In [15]:
class New_model_ft(nn.Module):
    def __init__(self):
        super().__init__()
        self.model_ft = model_ft
        for param in self.model_ft.features.parameters():
          param.requires_grad = False
        self.fc = nn.Linear(1792, 11)
    
    def forward(self, x):
        # forward pass сети
        # умножение на матрицу весов 1 слоя и применение функции активации
        x = self.model_ft(x)
        x = self.fc(x)
        return x

In [17]:
net = New_model_ft().to(device)
model_ft1 = train_model(net , criterion , optimizer_ft ,exp_lr_scheduler , num_epochs=10)

Epoch 0/9
----------
train Loss: 2.4173 Acc: 0.0861
val Loss: 2.4092 Acc: 0.0942

Epoch 1/9
----------
train Loss: 2.4185 Acc: 0.0844
val Loss: 2.4078 Acc: 0.0933

Epoch 2/9
----------
train Loss: 2.4203 Acc: 0.0860
val Loss: 2.4097 Acc: 0.0923

Epoch 3/9
----------
train Loss: 2.4177 Acc: 0.0836
val Loss: 2.4085 Acc: 0.0942

Epoch 4/9
----------
train Loss: 2.4212 Acc: 0.0842
val Loss: 2.4103 Acc: 0.0979

Epoch 5/9
----------
train Loss: 2.4198 Acc: 0.0854
val Loss: 2.4114 Acc: 0.0868

Epoch 6/9
----------
train Loss: 2.4186 Acc: 0.0851
val Loss: 2.4103 Acc: 0.0877

Epoch 7/9
----------
train Loss: 2.4191 Acc: 0.0830
val Loss: 2.4099 Acc: 0.0997

Epoch 8/9
----------
train Loss: 2.4208 Acc: 0.0841
val Loss: 2.4108 Acc: 0.0896

Epoch 9/9
----------
train Loss: 2.4210 Acc: 0.0824
val Loss: 2.4097 Acc: 0.0868

Training complete in 5m 56s
Best val Loss: 2.407774


In [5]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=20)
torch.save(model_ft,'resnet18-bestloss.pth')
torch.save(model_ft.state_dict(), "weights-resnet18-bestloss.pth")
run.stop()

Epoch 0/19
----------
train Loss: 1.3654 Acc: 0.6292
val Loss: 0.5724 Acc: 0.8116

Epoch 1/19
----------
train Loss: 0.6962 Acc: 0.7756
val Loss: 0.5359 Acc: 0.8449

Epoch 2/19
----------
train Loss: 0.5339 Acc: 0.8243
val Loss: 0.3844 Acc: 0.8753

Epoch 3/19
----------
train Loss: 0.4055 Acc: 0.8651
val Loss: 0.5644 Acc: 0.8467

Epoch 4/19
----------
train Loss: 0.5062 Acc: 0.8383
val Loss: 0.3910 Acc: 0.8707

Epoch 5/19
----------
train Loss: 0.3886 Acc: 0.8723
val Loss: 0.3133 Acc: 0.8947

Epoch 6/19
----------
train Loss: 0.3215 Acc: 0.8935
val Loss: 0.2940 Acc: 0.9049

Epoch 7/19
----------
train Loss: 0.2177 Acc: 0.9235
val Loss: 0.2629 Acc: 0.9151

Epoch 8/19
----------
train Loss: 0.1807 Acc: 0.9398
val Loss: 0.2690 Acc: 0.9132

Epoch 9/19
----------
train Loss: 0.1777 Acc: 0.9365
val Loss: 0.2771 Acc: 0.9104

Epoch 10/19
----------
train Loss: 0.1660 Acc: 0.9418
val Loss: 0.2786 Acc: 0.9160

Epoch 11/19
----------
train Loss: 0.1496 Acc: 0.9491
val Loss: 0.2901 Acc: 0.9086

Ep

In [20]:
model_efinet = copy(model_ft)

TypeError: 'module' object is not callable

In [9]:
# import gc
# torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=None, abbreviated=False)
# gc.collect()

124

In [10]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:8192"

In [21]:
run = neptune.init_run(
    project="neas1231/Neas1231",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmY2VjMGIzOS01ZjI1LTQ1MTItODQxYi0zMjIyOWIwYWI0MzIifQ==",
)
run["config/data_dir"] = data_dir
model_conv = torchvision.models.efficientnet_b4(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.classifier[1].in_features

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.Adam(model_conv.classifier[1].parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

https://app.neptune.ai/neas1231/Neas1231/e/NEAS-66


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000002119BDEB670>
Traceback (most recent call last):
  File "D:\hroneniyas\anacondas\envs\gputorch\lib\site-packages\torch\utils\data\dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "D:\hroneniyas\anacondas\envs\gputorch\lib\site-packages\torch\utils\data\dataloader.py", line 1437, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


In [22]:
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler,
                       num_epochs=20)
torch.save(model_conv,'conv-resnet18-bestloss.pth')
torch.save(model_conv.state_dict(), "conv-weights-resnet18-bestloss.pth")

Epoch 0/19
----------
train Loss: 0.1935 Acc: 0.9336
val Loss: 0.2809 Acc: 0.9123

Epoch 1/19
----------
train Loss: 0.1767 Acc: 0.9383
val Loss: 0.2681 Acc: 0.9160

Epoch 2/19
----------
train Loss: 0.1845 Acc: 0.9344
val Loss: 0.2670 Acc: 0.9104

Epoch 3/19
----------
train Loss: 0.1931 Acc: 0.9344
val Loss: 0.2689 Acc: 0.9178

Epoch 4/19
----------
train Loss: 0.1918 Acc: 0.9331
val Loss: 0.2897 Acc: 0.9067

Epoch 5/19
----------
train Loss: 0.1948 Acc: 0.9345
val Loss: 0.2819 Acc: 0.9095

Epoch 6/19
----------
train Loss: 0.1907 Acc: 0.9341
val Loss: 0.2867 Acc: 0.9095

Epoch 7/19
----------
train Loss: 0.1987 Acc: 0.9297
val Loss: 0.2749 Acc: 0.9141

Epoch 8/19
----------
train Loss: 0.1867 Acc: 0.9354
val Loss: 0.2794 Acc: 0.9114

Epoch 9/19
----------
train Loss: 0.1856 Acc: 0.9360
val Loss: 0.2778 Acc: 0.9132

Epoch 10/19
----------
train Loss: 0.1888 Acc: 0.9338
val Loss: 0.2801 Acc: 0.9077

Epoch 11/19
----------
train Loss: 0.1864 Acc: 0.9337
val Loss: 0.2820 Acc: 0.9049

Ep